# 05 - AI Functions 与多模态

## 学习目标

- 了解 Daft 内置的 AI Functions
- 掌握文本分类（`classify_text`）
- 掌握文本嵌入（`embed_text`）和语义搜索（`cosine_distance`）
- 学会使用 LLM 进行结构化数据提取（`prompt`）

> **注意**：本 Notebook 的 AI Functions 需要 OpenAI API Key。请在运行前设置环境变量：
> ```bash
> export OPENAI_API_KEY='your-key-here'
> ```

In [ ]:
import os

import daft
from daft import col

print(f"Daft 版本: {daft.__version__}")

# 检查 OpenAI API Key
if os.environ.get("OPENAI_API_KEY"):
    print("OpenAI API Key 已配置")
else:
    print("警告：未设置 OPENAI_API_KEY 环境变量，AI Functions 相关 cell 将无法运行")
    print("请运行: export OPENAI_API_KEY='your-key-here'")

In [ ]:
# 读取产品数据
df = daft.read_parquet("../data/products.parquet")
df.select("name", "category", "description").show(5)

## 1. 文本分类（classify_text）

Daft 内置 `classify_text` 函数，支持零样本文本分类。无需训练模型，只需提供候选标签即可。

我们用产品名称来预测类别，然后与实际 `category` 列对比。

In [ ]:
from daft.functions import classify_text

# 取 20 条数据进行分类（控制 API 调用量）
df_sample = df.select("name", "category").limit(20)

# 获取所有实际类别作为候选标签
categories = [row["category"] for row in df.select("category").distinct().collect().to_pylist()]
print(f"候选标签: {categories}")

In [ ]:
# 使用 classify_text 进行零样本分类
df_classified = df_sample.with_column(
    "predicted_category",
    classify_text(
        col("name"),
        labels=categories,
        provider="openai",
        model="text-embedding-3-small",
    ),
)

df_classified.show()

In [ ]:
# 对比预测结果与实际类别
result = df_classified.collect()
rows = result.to_pylist()

correct = sum(1 for r in rows if r["category"] == r["predicted_category"])
total = len(rows)
print(f"分类准确率: {correct}/{total} = {correct/total:.1%}")

## 2. 文本嵌入（embed_text）

嵌入（Embedding）是将文本转换为高维向量的过程，语义相近的文本在向量空间中距离更近。

Daft 的 `embed_text` 函数可以直接在 DataFrame 上批量生成嵌入。

In [ ]:
from daft.functions import embed_text

# 取 20 条产品，为名称生成嵌入
df_products = df.select("product_id", "name", "category", "price").limit(20)

df_embedded = df_products.with_column(
    "embedding",
    embed_text(
        col("name"),
        provider="openai",
        model="text-embedding-3-small",
    ),
)

# 查看 schema，注意 embedding 列的类型
print("Schema:")
print(df_embedded.schema())

In [ ]:
# 查看嵌入结果
df_embedded.show(5)

## 3. 语义搜索（cosine_distance）

有了嵌入向量，就可以通过计算余弦距离来实现语义搜索。

流程：
1. 为查询文本生成嵌入
2. 与产品嵌入做 cross join
3. 计算余弦距离（越小越相似）
4. 按距离排序，取 Top-N

In [ ]:
from daft.functions import cosine_distance

# 构造查询
query_text = "高性价比的电子产品"

df_query = daft.from_pydict({"query": [query_text]})
df_query = df_query.with_column(
    "query_embedding",
    embed_text(
        col("query"),
        provider="openai",
        model="text-embedding-3-small",
    ),
)

# Cross join：查询 × 所有产品
df_search = df_query.join(df_embedded, how="cross")

# 计算余弦距离
df_search = df_search.with_column(
    "distance",
    cosine_distance(col("query_embedding"), col("embedding")),
)

# 按距离排序，展示最相似的 5 个产品
df_search.sort("distance").select("query", "name", "category", "price", "distance").show(5)

## 4. LLM 结构化提取（prompt）

Daft 的 `prompt` 函数可以直接调用 LLM，并通过 Pydantic Model 定义结构化输出格式。

我们从产品名称和描述中提取结构化信息：目标受众、核心卖点、情感倾向。

In [ ]:
from pydantic import BaseModel, Field
from daft.functions import prompt, format


# 定义结构化输出模型
class ProductInsight(BaseModel):
    target_audience: str = Field(description="目标受众，如：年轻人、家庭、专业人士")
    key_feature: str = Field(description="核心卖点，一句话概括")
    sentiment: str = Field(description="产品描述的情感倾向：正面/中性/负面")

In [ ]:
# 取 5 条数据（控制 API 调用成本）
df_for_llm = df.select("name", "category", "description").where(col("description").not_null()).limit(5)

# 构造 prompt 并调用 LLM
df_insights = df_for_llm.with_column(
    "insight",
    prompt(
        format(
            "分析以下产品信息，提取目标受众、核心卖点和情感倾向。\n产品名称: {}\n类别: {}\n描述: {}",
            col("name"),
            col("category"),
            col("description"),
        ),
        model="gpt-4o-mini",
        provider="openai",
        return_format=ProductInsight,
    ),
)

df_insights.show()

In [ ]:
# 展开结构化字段为独立列
from daft.functions import unnest

df_expanded = df_insights.select(
    "name",
    "category",
    unnest(col("insight")),
)

df_expanded.show()

## 总结

本节学习了 Daft 的 AI Functions：

| 函数 | 用途 | 说明 |
|------|------|------|
| `classify_text()` | 文本分类 | 零样本分类，无需训练 |
| `embed_text()` | 文本嵌入 | 生成向量表示 |
| `cosine_distance()` | 语义搜索 | 计算向量间余弦距离 |
| `prompt()` | LLM 调用 | 支持结构化输出（Pydantic） |
| `format()` | Prompt 模板 | 动态构造提示词 |
| `unnest()` | 展开 Struct | 将结构体字段展开为独立列 |

Daft 还支持更多 AI 能力：
- `embed_image()` - 图像嵌入
- `classify_image()` - 图像分类
- `decode_image()` / `download()` - 多模态数据处理
- 多种 Provider：OpenAI、Google、vLLM、LM Studio 等

## 练习题

1. 修改语义搜索的查询文本，尝试搜索"适合送礼的商品"，观察结果变化
2. 使用 `prompt` 为产品生成一句营销文案（提示：修改 Pydantic Model）
3. 对比不同嵌入模型（如 `text-embedding-3-large`）的搜索效果

## 恭喜完成 Demo 1！

你已经掌握了 Daft 的基础使用和 AI 能力。继续学习 [Demo 2: Ray on Kubernetes](../../demo2_ray_kubernetes/) 了解分布式计算。